In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [2]:
pip install langchain openai langchain-openai langchain-experimental langchainhub

Defaulting to user installation because normal site-packages is not writeable

   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2/3 [langchain-experimental]
   -------------------------- ------------- 2

## Agents 사용 예

In [ ]:
# LangChain 전체 패키지 import
# (에이전트, 도구, 체인 등 AI 자동화 기능 제공)
import langchain

# LangChain 내부 동작 과정을 콘솔에 모두 출력하도록 설정
# -> 에이전트의 생각, 도구 선택, 실행 과정이 전부 보임
langchain.verbose=True


In [ ]:
# LangChain Hub (프롬프트 템플릿 저장소) 불러오기
from langchain_classic import hub
# Agent 관련 핵심 기능들 import
from langchain_classic.agents import (AgentExecutor,  # 에이전트를 실제로 실행하는 실행기
                                      create_react_agent, # ReAct(생각+행동) 방식 에이전트 생성 함수
                                      load_tools)  # 에이전트가 사용할 도구를 불러오는 함수
# OpenAI GPT 모델을 LangChain에서 사용하기 위한 클래스
from langchain_openai import ChatOpenAI

# =========================
# 1. LLM (GPT 모델) 생성
# =========================

llm=ChatOpenAI(model_name='gpt-4.1-mini', # 사용할 OpenAI 모델 이름
               temperature=0)  # 출력 랜덤성 제거 (항상 일정한 답변)

# temperature=0 의미:
# -> 자동화 작업에서는 "창의성"보다 "일관성"이 중요
# -> 같은 입력 = 같은 결과


# =========================
# 2. 도구(Tool) 불러오기
# =========================

tools=load_tools(["terminal"],  # 사용할 도구 이름: terminal (터미널 명령 실행)
                 allow_dangerous_tools=True)  # 위험할 수 있는 도구 사용 허용


# terminal 도구가 하는 일:
# -> OS 명령 실행 가능
# -> ls, dir, cat, pwd 같은 실제 터미널 명령 수행

# allow_dangerous_tools=True 의미:
# -> 파일 삭제, 시스템 명령 같은 위험 가능성도 허용
# -> 실서비스에서는 매우 조심해야 함


# =========================
# 3. ReAct 프롬프트 불러오기
# =========================
prompt=hub.pull("hwchase17/react")



# hub.pull 의미:
# -> LangChain 공식 프롬프트 저장소에서 템플릿 다운로드

# "hwchase17/react":
# -> ReAct 에이전트용 표준 프롬프트
# -> GPT에게 "생각 -> 행동 -> 결과 -> 반복" 규칙을 알려줌


# =========================
# 4. ReAct 에이전트 생성
# =========================

agent=create_react_agent(llm, # 사용할 GPT 모델
                         tools,  # 사용할 도구 목록
                         prompt)  # ReAct 행동 규칙 프롬프트

# 이 한 줄에서 일어나는 일:

# GPT 모델 +
# 터미널 도구 +
# ReAct 사고 규칙

# => "스스로 판단해서 도구를 사용할 수 있는 AI 에이전트" 생성


# =========================
# 5. 에이전트 실행기 생성
# =========================
agent_executor=AgentExecutor(agent=agent, # 위에서 만든 에이전트
                             tools=tools,  # 사용할 도구 목록
                             verbose=True) # 실행 과정 로그 출력

# AgentExecutor 역할:
# -> 에이전트를 실제로 "실행"하는 컨트롤러

# 쉽게 말하면:
# agent = 두뇌
# executor = 운전사


# =========================
# 6. 에이전트 실행
# =========================
result=agent_executor.invoke({"input":"sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할때 인코딩도 알아서 맞춰줘"})


# invoke() 의미:
# -> 에이전트에게 실제 작업 명령

# input:
# -> 사람이 자연어로 입력한 요청


# =========================
# 실행 흐름 내부 동작
# =========================

# 1) GPT가 생각
# -> "파일 목록 확인 필요"

# 2) 터미널 도구 선택
# -> ls sample_data

# 3) 실제 OS 명령 실행

# 4) 결과 수신

# 5) GPT가 결과를 자연어로 정리

# result 변수에는:
# {
#   "input": 원래 질문,
#   "output": 최종 답변
# }
# 형태로 저장됨



> Entering new AgentExecutor chain...
Thought: sample_data 디렉터리에 있는 파일 목록을 확인하기 위해 터미널 명령어를 실행해야 한다. Windows 환경이므로 인코딩 문제를 고려하여 명령어를 실행하고 결과를 확인하겠다.
Action: terminal
Action Input: chcp 65001Executing command:
 chcp 65001
Active code page: 65001


C:\Users\User\AppData\Roaming\Python\Python313\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Thought: 현재 코드 페이지를 UTF-8(65001)로 변경하여 인코딩 문제를 방지했다. 이제 sample_data 디렉터리의 파일 목록을 확인하는 명령어를 실행하겠다.
Action: terminal
Action Input: dir sample_dataExecuting command:
 dir sample_data
 Volume in drive C has no label.
 Volume Serial Number is 6C6F-4BFC

 Directory of c:\Sesac_Workspace\sesac_LLM_Workspace\sample_data

2026-01-21  오전 10:14    <DIR>          .
2026-01-21  오전 10:14    <DIR>          ..
2026-01-02  오후 03:13            23,105 diabetes.csv
2026-01-07  오후 05:00             4,466 iris3.csv
2025-11-21  오후 03:48            11,829 notExercise.xlsx
2025-12-26  오전 11:33               479 scores_em.csv
2025-12-26  오전 11:29               233 sport_test.csv
2026-01-08  오후 01:49           361,279 wine.csv
               6 File(s)        401,391 bytes
               2 Dir(s)  831,043,739,648 bytes free
Final Answer: sample_data 디렉터리에는 다음과 같은 파일들이 있습니다.
- diabetes.csv
- iris3.csv
- notExercise.xlsx
- scores_em.csv
- sport_test.csv
- wine.csv

> Finished chain.


In [ ]:
# LangChain에서 Tool 클래스를 불러온다
# Tool은 "파이썬 함수 → AI가 사용할 수 있는 도구"로 바꿔주는 역할
from langchain_classic.tools import Tool

# ============================
# 1. AI가 사용할 함수 정의
# ============================

def my_super_func(param):
    # param : AI가 전달하는 입력값 (문자열 등)
    # 지금 예제에서는 사용하지 않지만,
    # 실제 서비스에서는 검색어, ID, 옵션 등이 들어올 수 있음
    
    # 항상 문자열 "42" 반환
    return "42"

# ============================
# 2. Tool 객체 생성
# ============================
tools=[
      # 파이썬 함수를 Tool 형태로 변환
    Tool.from_function(
        # 실제 실행될 파이썬 함수
        func=my_super_func,
        # AI가 인식하는 도구 이름
        # (ReAct 에이전트가 Action 단계에서 이 이름을 사용)
        name='The Answer',
        # AI가 "언제 이 도구를 써야 하는지" 판단하는 설명
        # 매우 중요: LLM은 description을 읽고 도구를 선택함
        description="생명, 우주, 그리고 모든것에 대한 궁극적인 질문의 답"
    )
]

In [ ]:
# 현재 tools 리스트에는:
# -> "The Answer" 라는 도구 하나만 등록된 상태


# ============================
# 3. ReAct 에이전트 생성
# ============================

agent=create_react_agent(llm,# 사용할 GPT 모델 (예: gpt-4.1-mini)
                         tools, # 위에서 만든 Tool 목록
                         prompt)# ReAct 사고 규칙 프롬프트 (hwchase17/react)

# 이 한 줄이 하는 일:

# GPT 모델
# + ReAct 프롬프트(생각 → 행동 → 결과 → 반복 규칙)
# + The Answer 도구

# => "스스로 판단해서 도구를 쓸 수 있는 AI 에이전트" 생성


# ============================
# 4. 에이전트 실행기 생성
# ============================
agent_executor=AgentExecutor(agent=agent, # 위에서 생성한 에이전트
                             tools=tools,# 사용할 도구 목록
                             verbose=True)  # 내부 동작 로그 출력 (생각, 행동, 결과 확인 가능)


# AgentExecutor 역할:
# -> 에이전트를 실제로 실행하는 관리자

# 비유:
# agent = 두뇌
# executor = 실행 엔진


# ============================
# 5. 에이전트 실행
# ============================

 # AI에게 전달할 자연어 명령
result=agent_executor.invoke({"input":"이 세계의 진리를 알려주세요"})


# ============================
# 내부에서 일어나는 실제 흐름
# ============================

# 1) GPT Thought (생각)
# -> "생명, 우주, 진리라는 표현이 있다"
# -> The Answer 도구를 사용해야겠다

# 2) Action (행동)
# -> The Answer(param="...") 호출

# 3) Observation (관찰)
# -> my_super_func 실행됨
# -> "42" 반환

# 4) Final Answer
# -> GPT가 "42"를 이용해 자연어 답변 생성


# ============================
# 6. 결과 출력
# ============================
print(result)



> Entering new AgentExecutor chain...
Question: 이 세계의 진리를 알려주세요
Thought: 이 세계의 진리에 대해 궁극적인 답을 얻기 위해 The Answer 도구를 사용해야겠다.
Action: The Answer
Action Input: 이 세계의 진리42Thought: The observation "42" is a reference to the famous answer to the ultimate question of life, the universe, and everything from Douglas Adams' "The Hitchhiker's Guide to the Galaxy." It symbolizes that the ultimate truth or answer is elusive or perhaps beyond simple explanation.
Final Answer: 이 세계의 진리에 대한 궁극적인 답은 "42"입니다.

> Finished chain.
{'input': '이 세계의 진리를 알려주세요', 'output': '이 세계의 진리에 대한 궁극적인 답은 "42"입니다.'}


In [ ]:
# OpenAI GPT 모델을 LangChain에서 사용하기 위한 클래스
from langchain_openai import ChatOpenAI
# 프롬프트(질문 템플릿)를 만들기 위한 클래스
from langchain_classic.prompts import PromptTemplate
# "프롬프트 + LLM" 을 묶어서 실행하는 체인 클래스
from langchain_classic import LLMChain



# ================================
# 1️⃣ 요약용 프롬프트 템플릿 만들기
# ================================

# GPT에게 줄 지시문(프롬프트)
# {input} 자리에 실제 텍스트가 들어가게 됨
summarize_template="""
아래의 글을 결론만 한마디로 요약해주세요.

{input}
"""
# PromptTemplate 객체 생성
# → 나중에 input 변수에 값을 넣으면 자동으로 프롬프트 완성됨
summarize_prompt=PromptTemplate(
    input_variables=['input'], # 사용할 변수 이름
    template=summarize_template, # 위에서 만든 프롬프트 문자열
)


# ================================
# 2️⃣ GPT 모델 객체 생성
# ================================
chat=ChatOpenAI(model_name="gpt-4.1-mini", # 사용할 OpenAI 모델
                temperature=0) # 0이면 랜덤성 최소 (항상 안정적인 출력)


# ================================
# 3️⃣ LLMChain 생성
# ================================

# LLMChain = "프롬프트 + GPT 모델" 묶음
# → 이 체인을 실행하면 GPT가 프롬프트에 맞게 자동 실행됨
summarize_chain=LLMChain(llm=chat,# 사용할 GPT 모델
                         prompt=summarize_prompt) # 사용할 프롬프트 템플릿



# ================================
# 4️⃣ Tool 생성 (Agent가 사용할 도구)
# ================================

tools=[
     # summarize_chain.run 함수를 Tool로 등록
    Tool.from_function(
        # 실제 실행될 함수
        # Agent가 이 Tool을 선택하면 summarize_chain.run() 실행됨
        func=summarize_chain.run,
        # Agent가 인식하는 도구 이름
        name='Summarizer',
        # Agent에게 "이 도구가 뭐하는지" 설명
        description='Text summarizer'
    )
]

C:\Users\User\AppData\Local\Temp\ipykernel_28536\436821684.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  summarize_chain=LLMChain(llm=chat,prompt=summarize_prompt)


In [ ]:
# ================================
# 5️⃣ ReAct Agent 생성
# ================================

# create_react_agent:
# → LLM이 "생각 → 도구선택 → 실행 → 결과분석" 구조로 행동하도록 만듦

agent=create_react_agent(llm, # 판단과 추론을 담당하는 LLM (보통 chat)
                         tools,# 사용할 수 있는 도구 목록
                         prompt)  # Agent 행동 규칙 프롬프트

# ================================
# 6️⃣ AgentExecutor 생성
# ================================

# AgentExecutor = Agent 실행기
# 실제로 Agent를 실행시켜주는 컨트롤러 역할
agent_executor=AgentExecutor(agent=agent,# 사용할 Agent
                             tools=tools, # Agent가 사용할 Tool 목록
                             verbose=True)  # 실행 과정 로그 전부 출력

# ================================
# 7️⃣ 요약할 원본 텍스트
# ================================
text="""
다음을 요약해 주세요.
안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다.
OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다.
저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다.
제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다.
하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다.
질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다.
"""

# ================================
# 8️⃣ Agent 실행
# ================================

# invoke() 실행 과정:
# 1. Agent가 input 텍스트 읽음
# 2. "요약 필요" 판단
# 3. Summarizer Tool 선택
# 4. summarize_chain.run 실행
# 5. GPT 요약 생성
# 6. 결과 반환
result=agent_executor.invoke({"input":text})# Agent 입력값 전달
# ================================
# 9️⃣ 결과 출력
# ================================
print(result)



> Entering new AgentExecutor chain...
Question: 
다음을 요약해 주세요.
안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다.
OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다.
저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다.
제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다.
하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다.
질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다.

Thought: 요약을 위해 주어진 텍스트의 핵심 내용을 간결하게 정리해야 한다.
Action: Summarizer
Action Input: 안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다. 제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다. 질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다.저는 다양한 주제에 대해 도움을 드릴 수 있는 AI 언어 모델입니다.Final Answer: 저는 OpenAI의 GPT-3.5 아키텍처 기반 AI 언어 모델 ChatGPT로, 자연어 이해와 생성에 능하며 2021년 9월까지의 정보를 바탕으로 다양한 질문에 답하고 대화를 나눌 수 있습니다.

> Finished chain.
{'input': '\n다음을 요약해 주세요.\n안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다.\nOpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다.\n저는 자연어 이해와

## Function calling을 사용하는 OpenAI Functions Agent


In [ ]:
# LangChain 에이전트에서 도구(tool)를 불러오는 함수
from langchain_classic.agents import load_tools
# Agent 실행기(관리자)와 OpenAI 함수 기반 Agent 생성 함수
from langchain_classic.agents import AgentExecutor,create_openai_functions_agent
# OpenAI(ChatGPT) 모델을 사용하기 위한 클래스
from langchain_openai import ChatOpenAI


# ===============================
# 1. GPT 모델 객체 생성
# ===============================

# ChatGPT 모델을 LangChain에서 사용하기 위한 객체 생성
# model='gpt-4.1-mini' : 사용할 OpenAI 모델 지정
# temperature=0 : 답변의 랜덤성 제거 (항상 최대한 정확하게 동작)
llm=ChatOpenAI(model='gpt-4.1-mini',temperature=0)

# ===============================
# 2. 사용할 Tool(도구) 불러오기
# ===============================

# terminal 도구를 로드
# → 실제 운영체제 터미널 명령어(ls, dir 등)를 실행할 수 있음
# allow_dangerous_tools=True
# → rm, del 같은 위험한 명령도 허용 (주의해서 사용해야 함)
tools=load_tools(['terminal'],allow_dangerous_tools=True)

# ===============================
# 3. Agent용 기본 프롬프트 불러오기
# ===============================

# LangChain Hub에서 Agent 전용 프롬프트 템플릿 다운로드
# 이 프롬프트에는:
# - 언제 도구를 써야 하는지
# - 결과를 어떻게 정리해서 말해야 하는지
# 규칙이 이미 정의되어 있음
prompt=hub.pull('hwchase17/openai-functions-agent')

# ===============================
# 4. Agent 생성
# ===============================

# GPT + Tools + Prompt 를 합쳐서
# "도구를 사용할 줄 아는 AI Agent" 생성
agent=create_openai_functions_agent(llm,# GPT 두뇌
                                    tools,# 터미널 도구
                                    prompt)# 행동 규칙 설명서

# ===============================
# 5. Agent 실행 관리자 생성
# ===============================

# AgentExecutor는 실제 실행을 담당하는 컨트롤러
# verbose=True :
# → Agent가 어떤 생각을 했는지
# → 어떤 도구를 썼는지
# → 어떤 명령을 실행했는지
# 전부 터미널에 출력해줌 (디버깅용)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)



# ===============================
# 6. Agent 실행 (사용자 명령 전달)
# ===============================

# invoke() 에 자연어 명령을 전달
# Agent가 스스로 판단해서:
# 1) 터미널이 필요한 작업인지 판단
# 2) OS 명령 생성 (ls 또는 dir)
# 3) 실제 실행
# 4) 결과를 다시 정리해서 출력
agent_executor.invoke({"input":"sample_data 디렉토리에 있는 파일 목록을 알려줘 터미널을 사용할때 인코딩도 알아서 맞춰서 해줘"})



> Entering new AgentExecutor chain...

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir


C:\Users\User\AppData\Roaming\Python\Python313\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', '&&', 'dir sample_data']}`


Executing command:
 ['chcp 65001', '&&', 'dir sample_data']
Parameter format not correct - 65001;

Invoking: `terminal` with `{'commands': ['chcp 65001 && dir sample_data']}`


Executing command:
 ['chcp 65001 && dir sample_data']
Active code page: 65001
 Volume in drive C has no label.
 Volume Serial Number is 6C6F-4BFC

 Directory of c:\Sesac_Workspace\sesac_LLM_Workspace\sample_data

2026-01-21  오전 10:14    <DIR>          .
2026-01-21  오전 10:14    <DIR>          ..
2026-01-02  오후 03:13            23,105 diabetes.csv
2026-01-07  오후 05:00             4,466 iri

{'input': 'sample_data 디렉토리에 있는 파일 목록을 알려줘 터미널을 사용할때 인코딩도 알아서 맞춰서 해줘',
 'output': 'sample_data 디렉토리에는 다음과 같은 파일들이 있습니다:\n- diabetes.csv\n- iris3.csv\n- notExercise.xlsx\n- scores_em.csv\n- sport_test.csv\n- wine.csv\n\n필요한 작업이 있으면 알려주세요!'}

In [11]:
pip install duckduckgo_search

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 3.1/3.1 MB 21.1 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [duckduckgo_search]
   -------------------- ------------------- 1/2 [duckduckgo_search]
   ---------------------------------------- 2/2 [duckduckgo_search]

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [16]:
pip install ddgs

Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/5 [socksio]
   ---------------- ----------------------- 2/5 [hpack]
   ---------------- ----------------------- 2/5 [hpack]
   ------------------------ --------------- 3/5 [h2]
   ------------------------ --------------- 3/5 [h2]
   -------------------------------- ------- 4/5 [ddgs]
   -------------------------------- ------- 4/5 [ddgs]
   -------------------------------- ------- 4/5 [ddgs]
   -------------------------------- ------- 4/5 [ddgs]
   -------------------------------- ------- 4/5 [ddgs]
   -------------------------------- ------- 4/5 [ddgs]
   ---------------------------------------- 5/5 [ddgs]

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# LangChain 라이브러리 전체를 불러옴
# (debug, verbose 같은 전역 옵션 설정용)
import  langchain

# =========================
# LangChain 로그 설정
# =========================

# 내부 동작을 아주 자세하게 출력 (API 요청, 내부 처리 과정 등)
# → 개발/디버깅 할 때 사용
langchain.debug=True

# LangChain 기본 요약 로그 출력 여부
# → 여기서는 끔 (아래 AgentExecutor에서 verbose 따로 켬)
langchain.verbose=False


In [ ]:
# =========================
# Agent 관련 기능 불러오기
# =========================

# 검색, 계산기 같은 Tool(도구)을 불러오는 함수
from langchain_classic.agents import load_tools
# Agent 실행 관리자 + OpenAI 함수 기반 Agent 생성 함수
from langchain_classic.agents import AgentExecutor,create_openai_functions_agent
# OpenAI(ChatGPT) 모델을 LangChain에서 사용하기 위한 클래스
from langchain_openai import ChatOpenAI


# =========================
# GPT 모델 생성
# =========================

# ChatGPT 모델 객체 생성
# model='gpt-4.1-mini' → 사용할 GPT 모델
# temperature=0 → 랜덤성 제거 (항상 최대한 정확한 답변)
llm=ChatOpenAI(model='gpt-4.1-mini',temperature=0)

# =========================
# 검색 Tool 불러오기
# =========================

# DuckDuckGo 검색 도구 로드
# → Agent가 인터넷 검색 가능해짐
tools=load_tools(['ddg-search'])

# =========================
# Agent 기본 프롬프트 불러오기
# =========================

# LangChain Hub에서 공식 Agent 프롬프트 다운로드
# → "언제 도구를 써야 하는지", "어떻게 행동할지" 규칙 포함
prompt=hub.pull("hwchase17/openai-functions-agent")


# =========================
# Agent 생성
# =========================

# GPT + 검색 Tool + 프롬프트를 합쳐서
# "검색할 줄 아는 AI Agent" 생성




agent=create_openai_functions_agent(llm,tools,prompt)

# =========================
# Agent 실행 관리자 생성
# =========================

# AgentExecutor는 Agent를 실제로 실행하는 관리자
# verbose=True → Agent가 생각하는 과정 출력
agent_executor=AgentExecutor(agent=agent, # GPT 두뇌
                             tools=tools, # 검색 도구
                             verbose=True)  # Agent 행동 설명서

# =========================
# Agent 실행 (질문 전달)
# =========================

# 사용자 질문 전달
# Agent가:
# 1) 검색이 필요한지 판단
# 2) DuckDuckGo 검색 실행
# 3) 결과 요약
# 4) 답변 생성
result=agent_executor.invoke({"input":"서울과 부산의 날씨를 알려줘"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': '서울 날씨'}`


2025. 12. 28. · 28∼29일 이틀 동안 경기 북동부와 강원 내륙·산지, 대전·세종·충남은 5㎜ 안팎의 비가 예보됐다. 29일 하루 서울·인천·경기와 서해5도, 충북은 5㎜ 안팎, 전북은 5㎜ 미만, ... 3시간 전 · 아침최저기온은 -13~1도, 낮최고기온은 -6~1도를 보이겠습니다. 바다의 물결은 서해상에서 1.0~4.0m, 남해상에서 0.5~4.0m, 동해상에서 1.0~4.0m로 전해상에서 높게 일겠 ... 22시간 전 · 절기 '대한'인 오늘, 올겨울 최강 한파가 본격화했습니다. 오늘 서울 아침 기온은 영하 11.8도, 칼바람에 체감온도는 영하 18도까지 곤두박질했는데요, 한낮에도 체감 ... 2025. 9. 16. · 종로구, 서울특별시 날씨 예보 · 아침. -11°. 강수확률0% · 오후. -7°. 강수확률0% · 저녁. -12°. 강수확률0% · 밤새. -12°. 강수확률1% ... 2025. 8. 30. · 서울의 14일 간 날씨 예보 · 체감 온도: -7.4°C - 1.1°C18.7°F - 33.9°F · 평균 습도: 52% · 강수 확률: 비:61% 눈:0%.
Invoking: `duckduckgo_search` with `{'query': '부산 날씨'}`


The Weather Channel 및 Weather.com이 제공하는 오늘과 오늘 밤 부산 광역시 일기예보, 날씨 상태 및 도플러 레이더 (46700) 부산광역시 강서구 대저로 63번길 54 T. 대표전화 051-718-0200 (평일 9:00~18:00, 야간휴일은 당직실 연결) | E-mail. webmasterkma@kma.go.kr 본 홈페이지에 게시된 이메일 주소가 자동 수집되는 것을 거부하며, 이를 위반시 정보통신망법에 의해 처벌됨을 유념하시기 ... Hourly weather fore

In [ ]:
# =========================
# 결과 출력
# =========================

# Agent가 최종 생성한 답변만 출력
print(result['output'])

현재 서울의 날씨는 아침 최저기온이 -13도에서 1도 사이이고, 낮 최고기온은 -6도에서 1도 사이입니다. 오늘 서울은 매우 추운 날씨로, 아침 기온은 영하 11.8도이며 체감온도는 영하 18도까지 떨어질 것으로 예상됩니다.

부산의 날씨 정보는 구체적인 온도는 제공되지 않았으나, 현재 부산의 일기예보와 시간별 기온, 일출 및 일몰 시간 등을 확인할 수 있습니다. 부산은 서울보다는 상대적으로 온화한 날씨일 가능성이 높습니다.

더 구체적인 부산의 현재 기온이나 날씨 상태가 필요하시면 알려주세요!


**OpenAI API는 functions를 더 이상 사용하지 않고 tools를 사용합니다. tools API를 사용하면 모델에서 여러 함수를 한 번에 호출하도록 요청할 수 있으므로 일부 아키텍처에서 응답 시간을 줄일 수 있습니다. OpenAI 모델에는 tools agent를 사용하는 것이 좋습니다**

In [ ]:
# ==============================
# 1. 필요한 라이브러리 가져오기
# ==============================

# Agent가 사용할 도구(tool)를 불러오는 함수
from langchain_classic.agents import load_tools

# Agent 실행기(AgentExecutor)와 OpenAI Tool 기반 Agent 생성 함수

from langchain_classic.agents import AgentExecutor,create_openai_tools_agent

# OpenAI Chat 모델을 사용하기 위한 클래스
from langchain_openai import ChatOpenAI


# ==============================
# 2. LLM (AI 모델) 생성
# ==============================

llm=ChatOpenAI(model='gpt-4.1-mini', # 사용할 OpenAI 모델 이름
               temperature=0)# 답변 랜덤성 (0 = 가장 정확하고 안정적)

# temperature=0 이유:
# → Agent는 "판단 + 도구 선택"이 중요
# → 창의성보다 정확성이 중요하기 때문


# ==============================
# 3. Agent가 사용할 Tool 불러오기
# ==============================
tools=load_tools(['ddg-search'])


# 'ddg-search' 의미:
# → DuckDuckGo 검색 엔진 사용
# → Agent가 인터넷 검색을 할 수 있게 됨

# tools 안에는 실제 검색 기능 객체가 들어 있음
# 예:
# tools = [DuckDuckGoSearchTool 객체]


# ==============================
# 4. Agent 전용 Prompt 불러오기
# ==============================
prompt=hub.pull("hwchase17/openai-tools-agent")

# hub.pull() :
# → LangChain Prompt Hub에서 미리 만들어진 프롬프트 다운로드

# 이 프롬프트 역할:
# - ReAct 구조 사용 방법
# - Tool 호출 규칙
# - 출력 형식
# 등을 Agent에게 알려주는 "설명서"


# ==============================
# 5. Agent 생성
# ==============================
agent=create_openai_tools_agent(llm,  # 사용할 AI 모델
                                tools, # 사용할 Tool 목록
                                prompt)# Agent용 프롬프트


# 이 한 줄에서 만들어지는 것:

# LLM + Tools + Prompt
#        ↓
#     OpenAI Tool Calling 기반 Agent 생성

# 이제 agent는:
# - 질문 분석 가능
# - Tool 선택 가능
# - Function Calling(JSON 명령) 생성 가능


# ==============================
# 6. AgentExecutor 생성
# ==============================
agent_executor=AgentExecutor(agent=agent,# 위에서 만든 Agent
                             tools=tools,# Agent가 사용할 Tool 목록
                             verbose=True) # 실행 과정 로그 출력

# AgentExecutor 역할:
# → Agent를 실제로 실행하는 "컨트롤러"

# verbose=True 효과:
# 실행 중 이런 로그가 보임:

# Thought: 무엇을 할지 생각
# Action: 어떤 Tool을 쓸지 결정
# Observation: Tool 실행 결과
# Final Answer: 최종 답변

# 즉, ReAct 과정이 전부 출력됨


# ==============================
# 7. Agent 실행 (질문 입력)
# ==============================

result=agent_executor.invoke({"input":"서울과 부산의 날씨를 알려줘"})



# invoke() :
# → Agent 실행 시작

# input 값:
# → Agent에게 전달할 사용자 질문

# 내부 동작 자동 수행:

# 1) 질문 분석
# 2) 날씨는 인터넷 검색 필요 판단
# 3) ddg-search Tool 선택
# 4) 검색 실행
# 5) 결과 분석
# 6) 최종 답변 생성


# ==============================
# 8. 결과 구조 설명 (출력 예)
# ==============================

# result 구조 예시:

# {
#   "input": "서울과 부산의 날씨를 알려줘",
#   "output": "서울은 25도 맑음, 부산은 23도 흐림",
#   "intermediate_steps": [...]
# }

# 최종 답변만 출력하고 싶으면:

# print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': '서울 날씨'}`


2025. 12. 28. · 28∼29일 이틀 동안 경기 북동부와 강원 내륙·산지, 대전·세종·충남은 5㎜ 안팎의 비가 예보됐다. 29일 하루 서울·인천·경기와 서해5도, 충북은 5㎜ 안팎, 전북은 5㎜ 미만, ... 3시간 전 · 아침최저기온은 -13~1도, 낮최고기온은 -6~1도를 보이겠습니다. 바다의 물결은 서해상에서 1.0~4.0m, 남해상에서 0.5~4.0m, 동해상에서 1.0~4.0m로 전해상에서 높게 일겠 ... 2025. 9. 16. · 종로구, 서울특별시 날씨 예보 · 아침. -11°. 강수확률0% · 오후. -7°. 강수확률0% · 저녁. -12°. 강수확률0% · 밤새. -12°. 강수확률1% ... 22시간 전 · 절기 '대한'인 오늘, 올겨울 최강 한파가 본격화했습니다. 오늘 서울 아침 기온은 영하 11.8도, 칼바람에 체감온도는 영하 18도까지 곤두박질했는데요, 한낮에도 체감 ... 2025. 8. 30. · 서울의 14일 간 날씨 예보 · 체감 온도: -7.4°C - 1.1°C18.7°F - 33.9°F · 평균 습도: 52% · 강수 확률: 비:61% 눈:0%.
Invoking: `duckduckgo_search` with `{'query': '부산 날씨'}`


2025. 11. 10. · 내일 · 시각: 01시 · 날씨: 구름 많음 · 기온(체감온도) -4℃(-10℃) · 체감온도: -10℃ · 강수량: - · - · 강수확률: 20% · 바람: 북서풍 약간 강 4m/s ... 2025. 5. 9. · 지금은 기상할 시간! 국립밀양기상박물관에 이어 2번째로 방문한 곳? 두구두구두구!!! 부산지방기상청입니다! 기상청 국민정책기자단 이종진 기자님, ... 2025. 12. 3. · 부산 시간별날씨 ; 04시 · 4.5m/s · 습도 20% ; 05시 · 5.

In [24]:
print(result['output'])

현재 서울과 부산의 날씨 정보는 다음과 같습니다.

서울: 현재 기온은 약 -13도에서 1도 사이이며, 아침 최저 기온은 -13도, 낮 최고 기온은 1도 정도입니다. 강수 확률은 낮고, 대체로 맑은 날씨가 예상됩니다.

부산: 현재 기온은 약 4도에서 17도 사이이며, 아침 최저 기온은 4도, 낮 최고 기온은 17도 정도입니다. 바람은 약간 불고, 대체로 맑은 날씨가 예상됩니다.

더 구체적인 시간대별 날씨나 추가 정보가 필요하시면 알려주세요.


In [ ]:
# ==============================
# JSON 처리를 위한 기본 라이브러리
# ==============================
import json
# → 파이썬에서 JSON 데이터를 다루기 위한 기본 모듈
# → 지금 코드에서는 직접 사용하지 않지만
#   결과를 파일로 저장하거나 변환할 때 자주 사용됨


# ==============================
# OpenAI LLM 연결 라이브러리
# ==============================
from langchain_openai import ChatOpenAI
# → OpenAI의 Chat 모델을 LangChain에서 쉽게 사용하게 해주는 클래스


# ==============================
# Extraction Chain 생성 함수
# ==============================

from langchain_classic.chains import create_extraction_chain
# → 텍스트에서 원하는 정보만 "구조화해서 추출"하는 체인을 생성하는 함수
# → 자연어 → JSON 변환 전용 도구


# ==============================
# 추출할 데이터 구조(Schema) 정의
# ==============================


schema={
     # properties:
    # → 어떤 필드를 추출할지 정의
    "properties":{
        # 사람 이름 (문자열)
        "person_name":{"type":"string"},
        # 사람 키 (정수 숫자)
        "person_height":{"type":"integer"},
        # 사람 머리 색 (문자열)
        "person_hair_color":{"type":"string"},
        # 강아지 이름 (문자열)
        "dog_name":{"type":"string"},
         # 강아지 품종 (문자열)
        "dog_breed":{"type":'string'},
    },
    # required:
    # → 반드시 포함되어야 하는 필드 지정
    "required":["person_name","person_height"]
}

# 이 Schema의 의미:
#
# AI에게 이렇게 말하는 것과 같음:
# "텍스트에서
#  사람 이름,
#  키,
#  머리색,
#  강아지 이름,
#  품종
#  을 찾아서 JSON 형태로 만들어라
#  그리고 이름과 키는 반드시 채워라"


# ==============================
# 분석할 원본 텍스트
# ==============================
text="""
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""


# 이 텍스트에는 다음 정보가 섞여 있음:
#
# Alex 키: 5 feet
# Claudia 키: Alex보다 1 feet 큼
# Claudia 머리색: brunette
# Alex 머리색: blonde
# Alex 강아지 이름: Frosty
# 강아지 품종: labrador


# ==============================
# OpenAI Chat 모델 생성
# ==============================
chat=ChatOpenAI(model='gpt-4.1-mini',# 사용할 OpenAI 모델
                temperature=0) # 랜덤성 제거 (정보 추출은 정확성이 중요)


# temperature=0 이유:
# → 창의적인 문장 생성이 아니라
# → 정확한 정보 추출이 목적이기 때문


# ==============================
# Extraction Chain 생성
# ==============================
chain=create_extraction_chain(schema,chat)


# 여기서 일어나는 일:
#
# schema + LLM(chat)
#        ↓
# "정보 추출 전용 파이프라인" 생성
#
# 이 체인은 내부적으로:
# - 프롬프트 생성
# - JSON Schema 적용
# - Function Calling 구조 사용
# 을 자동으로 처리함


# ==============================
# Chain 실행 (텍스트 분석 시작)
# ==============================
people=chain.invoke(text)


# invoke():
# → 체인 실행 함수
#
# 실행 흐름:
#
# 1) 텍스트를 LLM에 전달
# 2) AI가 글을 읽음
# 3) Schema 구조에 맞게 정보 추출
# 4) JSON 형태 결과 반환


# ==============================
# 결과 형태 예시
# ==============================

# people 변수에는 보통 아래와 같은 형태가 들어 있음:

# [
#   {
#     "person_name": "Alex",
#     "person_height": 5,
#     "person_hair_color": "blonde",
#     "dog_name": "Frosty",
#     "dog_breed": "labrador"
#   }
# ]

# 리스트 형태인 이유:
# → 여러 사람 정보가 동시에 추출될 수 있기 때문

In [ ]:
# people 파이썬 객체를
# JSON 문자열 형태로 변환한 뒤
# 사람이 보기 좋게 들여쓰기해서
# 콘솔에 출력한다
print(json.dumps(people, # JSON으로 바꿀 파이썬 객체
                 indent=2))# 들여쓰기 2칸 적용 (가독성 향상)

{
  "input": "\nAlex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.\nAlex's dog Frosty is a labrador and likes to play hide and seek.\n",
  "text": [
    {
      "person_name": "Alex",
      "person_height": 5,
      "person_hair_color": "blonde"
    },
    {
      "person_name": "Claudia",
      "person_height": 6,
      "person_hair_color": "brunette"
    },
    {
      "person_name": "Frosty",
      "dog_breed": "labrador"
    }
  ]
}


In [ ]:
# ==============================
# OpenAI Chat 모델 사용을 위한 클래스 불러오기
# ==============================
from langchain_openai import ChatOpenAI
# → OpenAI의 ChatGPT 모델을 LangChain 환경에서 사용하기 위한 클래스

# ==============================
# LLM 평가(Evaluation) 도구 불러오기
# ==============================
from langchain_classic.evaluation import load_evaluator
# → LLM 답변을 자동으로 평가(채점)하는 Evaluator 로더 함수


# ==============================
# 평가에 사용할 LLM 생성
# ==============================
chat=ChatOpenAI(model='gpt-4.1-mini',# 평가를 수행할 OpenAI 모델
                temperature=0)# 랜덤성 제거 (평가는 정확성이 중요)


# temperature=0 이유:
# → 평가 결과가 매번 달라지면 안 됨
# → 항상 일관된 채점 결과를 얻기 위해 0으로 설정


# ==============================
# QA(Question Answering) 평가기 생성
# ==============================
evaluator=load_evaluator("qa",   # QA(질문-답변) 평가 타입
                         eval_llm=chat) # 평가를 수행할 LLM 지정

# "qa" evaluator 의미:
# → 질문(input)
# → 모델 답변(prediction)
# → 정답(reference)
# 을 비교해서
# 답이 맞는지 판단하는 평가기

# 내부적으로 LLM에게 다음과 같은 역할을 시킴:
# "이 답변이 정답과 의미적으로 일치하는지 판단해라"


# ==============================
# 실제 평가 실행
# ==============================
result=evaluator.evaluate_strings(
     # --------------------------
    # 문제 (질문)
    # --------------------------
    input="""나는 시장에 가서 사과 10개를 샀어. 사과 2개를 이웃에게 주고, 2개를 수리공에게 주었어. 그리고 사과 5개를 더 사서 1개는 내가 먹었어. 나는 몇 개의 사과를 가지고 있었니?""",

    # input 의미:
    # → 원본 문제 텍스트
    # → 평가 기준이 되는 질문


    # --------------------------
    # 모델이 생성한 답변
    # --------------------------
    prediction="""먼저 사과 10개로 시작했어.
이웃에게 2개, 수리공에게 2개를 나누어 주었으므로 사과가 6개가 남았어.
그런 다음 사과 5개를 더 사서 이제 사과가 11개가 되었어.
마지막으로 사과 1개를 먹었으므로 사과 10개가 남게 돼.""",

    # prediction 의미:
    # → LLM(또는 시스템)이 생성한 답변
    # → "학생 답안지" 같은 역할


    # --------------------------
    # 실제 정답
    # -------------------------
    reference="10개",
     # reference 의미:
    # → 사람이 준비한 정답 데이터
    # → 채점 기준
)

In [ ]:
print(result)
# ==============================
# 결과(result) 구조 예시
# ==============================

# result 안에는 보통 이런 정보가 들어 있음:

# {
#   "score": 1,
#   "value": "CORRECT",
#   "reasoning": "The prediction correctly matches the reference answer."
# }

# score:
# 1 → 정답
# 0 → 오답

# value:
# "CORRECT" 또는 "INCORRECT"

# reasoning:
# 왜 맞았는지 또는 틀렸는지 LLM이 설명

{'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}
